In [23]:
import pandas as pd
from collections import Counter
from pyecharts import options as opts
from pyecharts.charts import Graph
import os

In [24]:
def check_common_strings_across_columns(df):
    """
    检查df的不同列中是否有相同的字符串，并返回列名和具体重复值
    参数：
    df（pd.DatdFrame）
    返回值：
    list：包含列名和重复值的列表
    """
    common_strings=[]
    columns=df.conlumns.tolist()

    for i in range(len(columns)):
        for j in range(i+1,len(columns)):
            col1=columns[i]
            col2=columns[j]
            col1_strings=set(df[col1].dropna().astype(str).tolist())
            col2_strings=set(df[col2].dropna().astype(str).tolist())
            common=col1_strings.intersection(col2_strings)
            if common:
                for value in common:
                    common_strings.append((col1,col2,value))
    return common_strings

In [25]:
def count_frequencies(df,column_name):
    """
    统计DataFrame中指定列中值的频次
    参数：
    df(pd.DataFrame):数据框
    column_name(str):需要统计的列名
    返回值：
    dict:各个值的频次
    """
    frenquency=Counter(df[column_name])
    return frenquency

In [26]:
def df_one_attribute_frequency(data,attribute_name):
    """
    # 统计频次并且按照频次值进行排序
    """
    frequency = dict(sorted(count_frequencies(data, attribute_name).items(), key=lambda item: item[1], reverse=True))
    return frequency

def df_two_attribute_frequency(data,attribute_name1,attribute_name2):
    """
    # 按attribute_name1分组，统计attribute_name2频次
    """
    temp_result = data.groupby(attribute_name1)[attribute_name2].apply(list).to_dict()
    frequency={key: dict(Counter(value)) for key, value in temp_result.items()}
    return frequency

In [27]:
def two_attribute_frequency_net(data,attribute1,attribute2,target_filepath,graph_title):
    attribute1_frequencies=df_one_attribute_frequency(data,attribute1)
    attribute2_frequencies=df_one_attribute_frequency(data,attribute2)
    attribute1_attribute2_frequencies=df_two_attribute_frequency(data,attribute1,attribute2)
    size_parameter=0.01
    # 创建节点和边
    nodes=[]
    attribute1_nodes = [{"name": key, "symbolSize": [size_parameter*value, 20], "symbol": "rect","category": 1} for key, value in attribute1_frequencies.items()]
    attribute2_nodes = [{"name": key, "symbolSize": [size_parameter*value, 20], "symbol": "rect","category": 2} for key, value in attribute2_frequencies.items()]

    # 计算每列的最大节点大小 
    # # 设置列间距和行间距
    max_attribute1_size = max(size_parameter*value for key, value in attribute1_frequencies.items()) if attribute1_frequencies else 0
    max_attribute2_size = max(size_parameter*value for key, value in attribute2_frequencies.items()) if attribute2_frequencies else 0
    column_spacing = max(max_attribute1_size, max_attribute2_size)
    row_spacing = 5

    # 设置节点的坐标
    nodes.append({"name": attribute1, "symbolSize": 30, "x": 0, "y": 0, "category": 0})
    for i, node in enumerate(attribute1_nodes):
        node["x"] = column_spacing
        node["y"] = i * row_spacing
        nodes.append(node)
    for i, node in enumerate(attribute2_nodes):
        node["x"] = column_spacing*2
        node["y"] = i * row_spacing
        nodes.append(node)

    num_rows_data = data.shape[0]
    links=[]
    attribute1_links = [{"source": attribute1, "target": key, "value": value/num_rows_data, "emphasis": {"label": {"show": True, "formatter": "频率: {c}"}}} for key, value in attribute1_frequencies.items()]
    attribute1_attribute2_links = [
        {"source": key, "target": sub_key, "value": sub_value/sum(value.values()), "emphasis": {"label": {"show": True,  "formatter": "频率: {c}"}}}
        for key, value in attribute1_attribute2_frequencies.items()
        for sub_key, sub_value in value.items()
    ]
    links += attribute1_links
    links += attribute1_attribute2_links

    # 定义分类
    categories = [
        {"name": "分类节点","itemStyle": {"color": "red", "fontSize": 14, "fontWeight": "bold"}},
        {"name": "缺陷类型","itemStyle": {"color": "blue", "fontSize": 14, "fontWeight": "bold"}},
        {"name": "处理方法类型","itemStyle": {"color": "blue", "fontSize": 14, "fontWeight": "bold"}}
    ]

    # 创建有向图
    graph = Graph()

    graph.add(
        "",
        nodes,
        links,
        categories=categories,
        layout="none",
        edge_label=opts.LabelOpts(is_show=False),
        is_draggable=True,
        edge_symbol=['none', 'arrow'],  # 设置边的形状为箭头
        edge_symbol_size=5,  # 设置箭头的大小
    )
    graph.set_global_opts(title_opts=opts.TitleOpts(title=graph_title))

    # 渲染图表
    graph.render(target_filepath)

# 指定CSV文件的路径
file_path = 'C:/Users/4097084/code/data_analysis/data/failed_data_csv/外修基础数据_202212.csv'
# 读取CSV文件
data = pd.read_csv(file_path)
# 替换重复值
data['缺陷'] = data['缺陷'].replace("others", "缺陷_Others")
data['修理方法'] = data['修理方法'].replace("others", "修理方法_Others")

two_attribute_frequency_net(data,"缺陷","修理方法","./test.html","【缺陷类型-修理方法】频率统计")

In [28]:
def process_csv_files(input_folder, output_folder, custom_name_part):
    # 获取输入文件夹中的所有CSV文件
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

    for csv_file in csv_files:
        file_path = os.path.join(input_folder, csv_file)
        data = pd.read_csv(file_path)

        # 替换重复值
        data['缺陷'] = data['缺陷'].replace("others", "缺陷_Others")
        data['修理方法'] = data['修理方法'].replace("others", "修理方法_Others")

        # 生成输出文件名
        output_file_name = f"{os.path.splitext(csv_file)[0]}_{custom_name_part}.html"
        output_file_path = os.path.join(output_folder, output_file_name)

        # 调用函数处理数据并生成图表
        two_attribute_frequency_net(data, "缺陷", "修理方法", output_file_path, f"【缺陷类型-修理方法】频率统计 \n {csv_file}")

# 示例用法
input_folder = r'C:/Users/4097084/code/data_analysis/data/failed_data_csv/'
output_folder = r'C:/Users/4097084/data/'
custom_name_part = 'frequency_analysis'

process_csv_files(input_folder, output_folder, custom_name_part)

In [ ]:
def multi_attribute_frequency_net(data, attributes, target_filepath, graph_title):
    size_parameter = 0.01
    nodes = []
    links = []
    column_spacing = 0
    row_spacing = 5
    num_rows_data = data.shape[0]

    # 创建节点和边
    for i, attribute in enumerate(attributes):
        frequencies = df_one_attribute_frequency(data, attribute)
        max_size = max(size_parameter * value for value in frequencies.values()) if frequencies else 0
        column_spacing = max(column_spacing, max_size)
        
        attribute_nodes = [{"name": key, "symbolSize": [size_parameter * value, 20], "symbol": "rect", "category": i + 1} for key, value in frequencies.items()]
        nodes.append({"name": attribute, "symbolSize": 30, "x": i * column_spacing, "y": 0, "category": 0})
        
        for j, node in enumerate(attribute_nodes):
            node["x"] = (i + 1) * column_spacing
            node["y"] = j * row_spacing
            nodes.append(node)
            links.append({"source": attribute, "target": node["name"], "value": frequencies[node["name"]] / num_rows_data, "emphasis": {"label": {"show": True, "formatter": "频率: {c}"}}})

    # 创建多列之间的连接
    for i in range(len(attributes) - 1):
        attribute1 = attributes[i]
        attribute2 = attributes[i + 1]
        frequencies = df_two_attribute_frequency(data, attribute1, attribute2)
        
        for key, value in frequencies.items():
            for sub_key, sub_value in value.items():
                links.append({"source": key, "target": sub_key, "value": sub_value / sum(value.values()), "emphasis": {"label": {"show": True, "formatter": "频率: {c}"}}})

    # 定义分类
    categories = [{"name": f"分类节点", "itemStyle": {"color": "red", "fontSize": 14, "fontWeight": "bold"}} ]+[{"name": f"分类{i+1}-{attributes[i]}", "itemStyle": {"color": "blue", "fontSize": 14, "fontWeight": "bold"}} for i in range(len(attributes))]

    # 创建有向图
    graph = Graph()
    graph.add(
        "",
        nodes,
        links,
        categories=categories,
        layout="none",
        edge_label=opts.LabelOpts(is_show=False),
        is_draggable=True,
        edge_symbol=['none', 'arrow'],
        edge_symbol_size=5,
    )
    graph.set_global_opts(title_opts=opts.TitleOpts(title=graph_title))

    # 渲染图表
    graph.render(target_filepath)
# 示例使用
file_path = 'C:/Users/4097084/code/data_analysis/data/failed_data_csv/外修基础数据_202212.csv'
data = pd.read_csv(file_path)
data['缺陷'] = data['缺陷'].replace("others", "缺陷_Others")
data['修理方法'] = data['修理方法'].replace("others", "修理方法_Others")

multi_attribute_frequency_net(data, ["缺陷", "修理方法"], "./test_multi.html", "【缺陷类型-修理方法】频率统计")